# CSE5ML_Lab_6B_LSTM for Text Generation
In this exercise, we are going to use a popular book from childhood as the dataset: Alice’s Adventures in Wonderland by Lewis Carroll. The dataset is downloaded from the online free book sources: https://www.gutenberg.org/ and named as wonderland.txt. After this practice, you can also download other free books and try build network and make predictions yourself.

In [1]:
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.
c:\Users\mikel\anaconda3\envs\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\mikel\anaconda3\envs\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\mikel\anaconda3\envs\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\mikel\anaconda3\envs\python37

### Load Data

In [ ]:
# load the text and covert to lowercase
filename = "wonderland.txt"
with open(filename, 'r', encoding='utf-8') as f:
    raw_text = f.read()
raw_text = raw_text.lower()

### Prepare data for RNN
Note: we cannot model the characters directly, instead we must convert the characters to integers.
This can be done easily by first creating a set of all of the distinct characters in the book, then creating a map of each character to a unique integer.

In [3]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [4]:
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '#': 4,
 '$': 5,
 '%': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '*': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '?': 27,
 '[': 28,
 ']': 29,
 '_': 30,
 'a': 31,
 'b': 32,
 'c': 33,
 'd': 34,
 'e': 35,
 'f': 36,
 'g': 37,
 'h': 38,
 'i': 39,
 'j': 40,
 'k': 41,
 'l': 42,
 'm': 43,
 'n': 44,
 'o': 45,
 'p': 46,
 'q': 47,
 'r': 48,
 's': 49,
 't': 50,
 'u': 51,
 'v': 52,
 'w': 53,
 'x': 54,
 'y': 55,
 'z': 56,
 'ù': 57,
 '—': 58,
 '‘': 59,
 '’': 60,
 '“': 61,
 '”': 62}

In [5]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  164045
Total Vocab:  63


Then, we will split the book text up into subsequences with a fixed length of 100 characters, an arbitrary length. Another way is to split the data up by sentences and pad the shorter sequences and truncate the longer ones.


Each training pattern of the network is comprised of 100 time steps of one character (X) followed by one character output (y). When creating these sequences, we slide this window along the whole book one character at a time, allowing each character a chance to be learned from the 100 characters that preceded it (except the first 100 characters of course).

In [6]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163945


Next we need to rescale the integers to the range 0-to-1 to make the patterns easier to learn by the RNN network. Then, we need to convert the output patterns (single characters converted to integers) into a one hot encoding. This is so that we can configure the network to predict the probability of each of the 47 different characters in the vocabulary (an easier representation) rather than trying to force it to predict precisely the next character.

In [7]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [8]:
X.shape

(163945, 100, 1)

### Build Model
You can either train the model yourself (take some time) or load the weights that I have provided on LMS.

In [9]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(X, y, epochs=20, batch_size=128)


Epoch 1/20


InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMM launch failed : a.shape=(128, 256), b.shape=(256, 1024), m=128, n=1024, k=256
	 [[{{node lstm_1/while/MatMul_1}}]]
	 [[Mean/_69]]
  (1) Internal: Blas GEMM launch failed : a.shape=(128, 256), b.shape=(256, 1024), m=128, n=1024, k=256
	 [[{{node lstm_1/while/MatMul_1}}]]
0 successful operations.
0 derived errors ignored.

### Generate text with the trained model

In [10]:
# pick a start of text to make predictions
start = 1468
pattern = dataX[start]
int_to_char = dict((i, c) for i, c in enumerate(chars))

print("start from")
print(''.join([int_to_char[value] for value in pattern]))

# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

start from
alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to


: 

: 

You can try build the following more complex LSTM model yourself and see if any change on the prediction.

In [11]:
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

# load text and covert to lowercase
filename = "wonderland.txt"
with open(filename, 'r', encoding='utf-8') as f:
    raw_text = f.read()
raw_text = raw_text.lower()

# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128)

model.save_weights('lstm_weights.h5')

# you can pick a random seed or give a specific start as we do before
# start = numpy.random.randint(0, len(dataX)-1)
start = 1468
pattern = dataX[start]
print("start from")
print(''.join([int_to_char[value] for value in pattern]))

# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Total Characters:  164045
Total Vocab:  63
Total Patterns:  163945
Epoch 1/20
1281/1281 [==============================] - 493s 381ms/step - loss: 2.9536
Epoch 2/20
1281/1281 [==============================] - 490s 382ms/step - loss: 2.6789
Epoch 3/20
1281/1281 [==============================] - 500s 391ms/step - loss: 2.5212
Epoch 4/20
1281/1281 [==============================] - 525s 410ms/step - loss: 2.3935
Epoch 5/20
1281/1281 [==============================] - 493s 385ms/step - loss: 2.2874
Epoch 6/20
1281/1281 [==============================] - 505s 394ms/step - loss: 2.2000
Epoch 7/20
1281/1281 [==============================] - 495s 386ms/step - loss: 2.1275
Epoch 8/20
1281/1281 [==============================] - 445s 347ms/step - loss: 2.0697
Epoch 9/20
1217/1281 [===========================>..] - ETA: 41:59 - loss: 2.0135